In [ ]:
import glob
from pathlib import Path
import numpy as np
import pandas as pd
import umap
import matplotlib.pyplot as plt

from livecellx.core.datasets import LiveCellImageDataset
from livecellx.sample_data import tutorial_three_image_sys

from livecellx.trajectory.feature_extractors import compute_haralick_features, compute_skimage_regionprops
from livecellx.preprocess.utils import normalize_img_to_uint8
from livecellx.core.parallel import parallelize
from livecellx.core.io_sc import prep_scs_from_mask_dataset
from livecellx.core.single_cell import create_sc_table
import livecellx.core.single_cell
from livecellx.core.single_cell import SingleCellStatic
from livecellx.core.io_utils import LiveCellEncoder


# dataset_dir_path = Path(
#     "../datasets/celltrackingchallenge/BF-C2DL-HSC/BF-C2DL-HSC/01"
# )

# mask_dataset_path = Path("../datasets/celltrackingchallenge/BF-C2DL-HSC/BF-C2DL-HSC/01_GT/TRA")

dataset_dir_path = Path(
    "../datasets/celltrackingchallenge/DIC-C2DH-HeLa/DIC-C2DH-HeLa/01"
)

# mask_dataset_path = Path("../datasets/celltrackingchallenge/DIC-C2DH-HeLa/DIC-C2DH-HeLa/01_GT/TRA")
# mask_dataset = LiveCellImageDataset(mask_dataset_path, ext="tif")
def extract_mask_time(filename):
    """seg_{time}.png -> time"""
    return int(filename.split("_")[1].split(".")[0])

mask_dataset_dir = Path(r"./application_results/celltrackingchallenge/hela_segmentation_cellpose_tn1/seg")
mask_time2url_paths = sorted(glob.glob(str((Path(mask_dataset_dir) / Path("*.png")))))
mask_time2url = {extract_mask_time(Path(path).name): path for path in mask_time2url_paths}


mask_dataset = LiveCellImageDataset(time2url=mask_time2url, ext="png")

time2url = sorted(glob.glob(str((Path(dataset_dir_path) / Path("*.tif")))))
time2url = {i: path for i, path in enumerate(time2url)}
img_dataset = LiveCellImageDataset(time2url=time2url, ext="tif")

In [ ]:
silver_mask_dir = Path("../datasets/celltrackingchallenge/DIC-C2DH-HeLa/DIC-C2DH-HeLa/01_ST/SEG")
silver_mask_dataset = LiveCellImageDataset(silver_mask_dir, ext="tif")

In [ ]:
len(img_dataset), len(mask_dataset)

In [ ]:
out_dir = Path("application_results/celltrackingchallenge/testing")
out_dir.mkdir(exist_ok=True, parents=True)

Compute the features  
Read the features in the next section if you already computed the features.

In [ ]:
from livecellx.core.io_sc import prep_scs_from_mask_dataset
scs = prep_scs_from_mask_dataset(mask_dataset, img_dataset)

In [ ]:
from livecellx.core.io_sc import prep_scs_from_mask_dataset
silver_seg_scs = prep_scs_from_mask_dataset(silver_mask_dataset, img_dataset)

In [ ]:
import importlib
import livecellx
importlib.reload(livecellx.trajectory.feature_extractors)
from livecellx.trajectory.feature_extractors import compute_haralick_features, compute_skimage_regionprops, parallelize_compute_features
from livecellx.preprocess.utils import normalize_img_to_uint8
from livecellx.core.parallel import parallelize

outputs = parallelize_compute_features(scs, compute_skimage_regionprops, dict(preprocess_img_func=normalize_img_to_uint8, sc_level_normalize=True))
features = outputs[0]
scs = outputs[1]

# # for sequential version without using parallelize
# for sc in scs:
#     features = compute_skimage_regionprops(sc, preprocess_img_func=normalize_img_to_uint8, sc_level_normalize=True)

In [ ]:
features = outputs[0]
scs = outputs[1]

In [ ]:
from typing import List
from livecellx.track.sort_tracker_utils import (
    gen_SORT_detections_input_from_contours,
    update_traj_collection_by_SORT_tracker_detection,
    track_SORT_bbox_from_contours,
    track_SORT_bbox_from_scs
)


sdata = track_SORT_bbox_from_scs(scs, img_dataset, mask_dataset=mask_dataset, max_age=0, min_hits=1)

In [ ]:
sdata.get_all_track_ids()

In [ ]:
silver_sdata = track_SORT_bbox_from_scs(silver_seg_scs, img_dataset, mask_dataset=silver_mask_dataset, max_age=0, min_hits=1)

In [ ]:
sdata_copy = track_SORT_bbox_from_scs(scs, img_dataset, mask_dataset=mask_dataset, max_age=0, min_hits=1)

In [ ]:
sct_operator.annotate_click_samples

In [ ]:
from livecellx.core.single_cell import SingleCellTrajectoryCollection, SingleCellTrajectory


SingleCellTrajectory.show_trajectory_on_grid(sdata_copy.get_trajectory(92), padding=100)

In [ ]:
def remove_empty_sc(self):
    remove_tids = []
    remove_scs = []
    for tid, sct in self:
        sc = sct.get_all_scs()[0]
        if len(sc.contour) == 0:
            remove_tids.append(tid)
            remove_scs.append(sc)
    for tid in remove_tids:
        self.pop_trajectory(tid)
remove_empty_sc(sdata)

In [ ]:
from livecellx.core.sc_seg_operator import create_sc_seg_napari_ui
from livecellx.core.sct_operator import create_sctc_edit_viewer_by_interval
from livecellx.core.single_cell import (
    SingleCellTrajectoryCollection,
    SingleCellStatic,
    SingleCellTrajectory,
    create_sctc_from_scs,
    filter_sctc_by_time_span,
)
import importlib
importlib.reload(livecellx.core.sct_operator)

tmp_sdata = create_sctc_from_scs(sdata.get_all_scs())
sct_operator = livecellx.core.sct_operator.create_sctc_edit_viewer_by_interval(tmp_sdata, img_dataset, span_interval=1000)

In [ ]:
len(sct_operator.cls2annotated_sample_infos["mitosis"])

In [ ]:
sc_annotation_labels = []
sc_annotation_labels_strs = []
for sc in sct_operator.shape_layer.properties["sc"]:
    sc_annotation_labels.append([])
    sc_annotation_labels_strs.append("")
    if "_annotation_label_info" not in sc.meta:
        continue
    for info in sc.meta["_annotation_label_info"]:
        sc_annotation_labels[-1].extend([info["label"]])
    sc_annotation_labels[-1] = list(set(sc_annotation_labels[-1]))
    annotation_repr = ",".join(sc_annotation_labels[-1])
    sc_annotation_labels_strs[-1] = annotation_repr

all_proerties = sct_operator.shape_layer.properties
all_proerties["annotation_labels"] = sc_annotation_labels_strs
sct_operator.shape_layer.properties = all_proerties
sct_operator.shape_layer.text = "annotation_labels"

In [ ]:
out_dir = Path("application_results/celltrackingchallenge/DIC-C2DH-HeLa/01_annotation")
out_dir.mkdir(exist_ok=True, parents=True)
sct_operator.save_annotations(out_dir)

In [ ]:
sct_operator.created_objects

In [ ]:
import napari
napari.__version__

In [ ]:
silver_sct_operator = livecellx.core.sct_operator.create_sctc_edit_viewer_by_interval(silver_sdata, img_dataset, viewer=sct_operator.viewer, span_interval=1000)

In [ ]:
from livecellx.core.io_sc import prep_scs_from_mask_dataset
scs = prep_scs_from_mask_dataset(mask_dataset, img_dataset)